In [8]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install PyQt5

Note: you may need to restart the kernel to use updated packages.


IMPORTING LIBRARIES

In [10]:
import speech_recognition as sr
import pyttsx3
import pywhatkit
import datetime
import wikipedia
import pyjokes
import requests
import json
import sys
import os
import subprocess
import imaplib
import email
import smtplib
import tempfile
import pyautogui
import pyperclip
import webbrowser
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

VOICE RECOGNIZER

In [11]:
listener = sr.Recognizer()

TEXT TO SPEECH ENGINE

In [12]:
engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty('voice', voices[1].id)  # voices[1] = female, voices[0] = male

In [13]:
def engine_talk(text):
    print('Vani:',text)
    try:
        engine.say(text)
        engine.runAndWait()
    except RuntimeError:
        print("Speech engine is already running. Skipping audio output.")

FEATURES

1. WEATHER FUNCTN

In [14]:
# ---------- Weather Function ----------
def weather(city):
    api_key = "4214d2473b3f1c79c03d0cbf06919b3c"
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}appid={api_key}&q={city}"
    
    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        main = data["main"]
        weather_desc = data["weather"][0]["description"]
        temp = main["temp"] - 273.15  # Convert from Kelvin to Celsius
        return f"The current temperature in {city} is {temp:.2f}°C with {weather_desc}."
    else:
        return "City not found."

2. NEWS FUNCTN 

In [15]:
def get_news():
    api_key = "a45e87f978784a71b6782f0763d66cd1"
    url = f"https://newsapi.org/v2/top-headlines?category=general&apiKey={api_key}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        articles = response.json().get("articles", [])[:5]
        news_summary = "Here are the top 5 global news headlines:\n"
        for i, article in enumerate(articles, 1):
            news_summary += f"{i}. {article.get('title', 'No Title')}\n"
        return news_summary.strip()
    except requests.exceptions.RequestException as e:
        return f"Failed to fetch news: {e}"

3. OPEN/CLOSE APPLICATIONS

In [16]:
# ---------- Open Application ----------
def open_app(app_name):
    app_paths = {
        "notepad": "notepad.exe",
        "calculator": "calc.exe",
        "chrome": "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe",
        "word": "C:\\Program Files\\Microsoft Office\\root\\Office16\\WINWORD.EXE",
        "excel": "C:\\Program Files\\Microsoft Office\\root\\Office16\\EXCEL.EXE",
        "weather":"weather.exe",
        "spotify":"spotify.exe"
    }

    for name, path in app_paths.items():
        if name in app_name:
            try:
                subprocess.Popen(path)
                return f"{name.capitalize()} is opening."
            except Exception as e:
                return f"Failed to open {name}: {str(e)}"
    return "Sorry, I couldn't find that application."


# ---------- Close Application ----------
def close_app(app_name):
    process_names = {
        "notepad": "notepad.exe",
        "calculator": "Calculator.exe",
        "chrome": "chrome.exe",
        "word": "WINWORD.EXE",
        "excel": "EXCEL.EXE"
    }

    for name, process in process_names.items():
        if name in app_name:
            os.system(f"taskkill /f /im {process}")
            return f"{name.capitalize()} has been closed."
    return "Application not found or not running."

4. OPEN FOLDERS

In [17]:
# ---------- Open Folder ----------
def open_folder(folder_name):
    folder_paths = {
        "downloads": os.path.expanduser('~\\Downloads'),
        "documents": os.path.expanduser('~\\Documents'),
        "desktop": os.path.expanduser('~\\Desktop'),
        "pictures": os.path.expanduser('~\\Pictures'),
        "music": os.path.expanduser('~\\Music'),
        "videos": os.path.expanduser('~\\Videos'),
        "recent": os.path.join(os.path.expanduser('~'), "AppData\\Roaming\\Microsoft\\Windows\\Recent"),
        "movie": os.path.expanduser('~\\MOVIE'),
        "sem 8": os.path.expanduser('~\\sem 8'),
    }

    folder_name = folder_name.lower()
    for key, path in folder_paths.items():
        if key in folder_name:
            if os.path.exists(path):
                os.startfile(path)
                return f"Opening {key} folder."
            else:
                return f"The {key} folder doesn't exist."
    
    # Try as direct path if no match
    if os.path.exists(folder_name):
        os.startfile(folder_name)
        return f"Opening {folder_name}."
    return "Sorry, I couldn't find that folder."

5. READ UNREAD EMAILS

In [18]:
def read_unread_emails(email_user, email_pass):
    try:
        mail = imaplib.IMAP4_SSL('imap.gmail.com')
        mail.login(email_user, email_pass)
        mail.select('inbox')
        status, response = mail.search(None, '(UNSEEN)')
        unread_ids = response[0].split()

        if not unread_ids:
            engine_talk("You have no unread emails.")
            return

        engine_talk(f"You have {len(unread_ids)} unread emails.")
        for e_id in unread_ids:
            _, msg_data = mail.fetch(e_id, '(RFC822)')
            for part in msg_data:
                if isinstance(part, tuple):
                    msg = email.message_from_bytes(part[1])
                    subject = msg['subject']
                    from_ = msg['from']
                    engine_talk(f"Email from {from_} with subject {subject}")
        mail.logout()
    except Exception as e:
        engine_talk("Sorry, I could not read your emails.")
        print("Email error:", e)



6. SEND AN EMAIL

In [19]:
def send_email(sender_email, sender_password, receiver_email, subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()

        return "Email sent successfully!"
    except Exception as e:
        print("Error sending email:", e)
        return "Sorry, I could not send the email."

7. TAKE A SCREENSHOT

In [20]:
def take_screenshot():
    try:
        folder = "Screenshots"
        if not os.path.exists(folder):
            os.makedirs(folder)

        filename = datetime.datetime.now().strftime("screenshot_%Y%m%d_%H%M%S.png")
        filepath = os.path.join(folder, filename)

        screenshot = pyautogui.screenshot()
        screenshot.save(filepath)
        subprocess.Popen(f'explorer "{os.path.abspath(folder)}"')

        return f"Screenshot saved as {filename} and folder opened."
    except Exception as e:
        return f"Failed to take screenshot: {str(e)}"

8. CREATE A STICKY NOTE

In [21]:
def create_sticky_note(note_content):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode='w', encoding='utf-8') as note_file:
            note_file.write(note_content)
            note_path = note_file.name
        subprocess.Popen(['notepad.exe', note_path])
        return "Reminder note created and opened on your desktop."
    except Exception as e:
        return f"Failed to create note: {str(e)}"

9. CHATGPT INTEGRATION

In [22]:
def ask_chatgpt(prompt):
    try:
        response = ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            provider=Yqcloud()
        )
        return response
    except Exception as e:
        print("gpt4free error:", e)
        return "Sorry, I couldn't connect to ChatGPT right now."


In [23]:
import g4f
print(g4f.Provider.__providers__)


[<class 'g4f.Provider.ARTA.ARTA'>, <class 'g4f.Provider.needs_auth.Anthropic.Anthropic'>, <class 'g4f.providers.base_provider.AsyncGeneratorProvider'>, <class 'g4f.providers.base_provider.AsyncProvider'>, <class 'g4f.Provider.template.BackendApi.BackendApi'>, <class 'g4f.providers.types.BaseProvider'>, <class 'g4f.Provider.needs_auth.BingCreateImages.BingCreateImages'>, <class 'g4f.Provider.hf_space.BlackForestLabs_Flux1Dev.BlackForestLabs_Flux1Dev'>, <class 'g4f.Provider.Blackbox.Blackbox'>, <class 'g4f.Provider.needs_auth.BlackboxPro.BlackboxPro'>, <class 'g4f.Provider.needs_auth.CablyAI.CablyAI'>, <class 'g4f.Provider.needs_auth.Cerebras.Cerebras'>, <class 'g4f.Provider.Chatai.Chatai'>, <class 'g4f.Provider.Cloudflare.Cloudflare'>, <class 'g4f.Provider.hf_space.CohereForAI_C4AI_Command.CohereForAI_C4AI_Command'>, <class 'g4f.Provider.Copilot.Copilot'>, <class 'g4f.Provider.needs_auth.CopilotAccount.CopilotAccount'>, <class 'g4f.providers.create_images.CreateImagesProvider'>, <class 

In [24]:
import g4f
from g4f.Provider import FreeGpt, Yqcloud, You

def ask_chatgpt(prompt):
    providers = [Yqcloud, FreeGpt, You]  # Use only reliable, no-auth ones

    for provider in providers:
        try:
            response = g4f.ChatCompletion.create(
                model="gpt-3.5-turbo",
                provider=provider(),
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            if response:
                return response
        except Exception as e:
            print(f"{provider.__name__} failed: {e}")
    
    return "Sorry, I couldn't connect to ChatGPT right now."


10. WHATSAPP MSGING

In [25]:
import webbrowser
import pyautogui
import pyperclip
import time
import subprocess
import platform

def send_whatsapp_message(contact_name, message):
    try:
        engine_talk(f"Opening WhatsApp Web to message {contact_name}")
        webbrowser.open("https://web.whatsapp.com")
        engine_talk("Bringing WhatsApp Web to front...")

        time.sleep(10)  # Allow it to load

        # Bring browser to front (Windows only)
        if platform.system() == "Windows":
            subprocess.run('powershell -command "(New-Object -ComObject Shell.Application).Windows() | ForEach-Object { $_.Visible = $true }"', shell=True)

        # Step 1: Click on the WhatsApp Search box (adjust x, y)
        pyautogui.click(x=354, y=272)  # Update these if needed
        time.sleep(1)

        # Step 2: Search contact
        pyperclip.copy(contact_name)
        pyautogui.hotkey("ctrl", "v")
        time.sleep(1)
        pyautogui.press("enter")
        time.sleep(1)

        # Step 3: Type and send message
        pyperclip.copy(message)
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press("enter")

        print(f" Message sent to: {contact_name}")
        engine_talk("Message sent successfully.")

    except Exception as e:
        print(" WhatsApp send error:", e)
        engine_talk("Sorry, I couldn't send the message.")


In [26]:
import pyautogui
print(pyautogui.position())

Point(x=1533, y=803)


USER COMMANDS

In [27]:
def user_commands(retries=2):
    for _ in range(retries):
        try:
            with sr.Microphone() as source:
                print("Start Speaking!!")
                voice = listener.listen(source, timeout=5, phrase_time_limit=8)
                command = listener.recognize_google(voice)
                command = command.lower()
                if 'vani' in command:
                    command = command.replace('vani', '')
                    engine_talk('yes, how can I help you?')
                print(f"You said: {command}")
                return command
        except sr.UnknownValueError:
            engine_talk("Sorry, I missed that. Could you please repeat?")
        except sr.RequestError as e:
            engine_talk("There was a problem connecting to the speech service.")
            print("Speech Recognition Error:", e)
            return None
    return None


In [28]:
def run_alexa():
    command = user_commands()

    if 'play' in command:
        song = command.replace('play', '')
        engine_talk('Playing ' + song)
        pywhatkit.playonyt(song)

    elif 'time' in command:
        current_time = datetime.datetime.now().strftime('%I:%M %p')
        engine_talk('The current time is ' + current_time)

    elif 'who is' in command:
        name = command.replace('who is', '')
        info = wikipedia.summary(name, 1)
        print(info)
        engine_talk(info)

    elif 'news' in command:
        news = get_news()
        print(news)
        engine_talk(news)

    elif 'joke' in command:
        engine_talk(pyjokes.get_joke())

    elif 'weather' in command:
        engine_talk("Please tell the name of the city.")
        city = user_commands()
        weather_info = weather(city)
        engine_talk(weather_info)

    elif 'open' in command:
        app = command.replace('open', '').strip()
        response = open_app(app)
        engine_talk(response)

    elif 'close' in command:
        app = command.replace('close', '').strip()
        response = close_app(app)
        engine_talk(response)

    elif 'stop' in command:
        engine_talk("Goodbye!")
        sys.exit()

    elif 'show' in command or 'directory' in command or 'file' in command:
        folder = command.replace('open folder', '').replace('open directory', '').replace('open file', '').strip()
        response = open_folder(folder)
        engine_talk(response)

    elif 'read unread emails' in command:
        engine_talk('Checking your unread emails...')
        read_unread_emails(EMAIL_USER, EMAIL_PASS)

    elif 'send email' in command:
        engine_talk('Please provide your email ID')
        sender_email = user_commands()
        engine_talk('Please provide your email password')
        sender_password = user_commands()
        engine_talk('Who is the receiver?')
        receiver_email = user_commands()
        engine_talk('What is the subject?')
        subject = user_commands()
        engine_talk('Please dictate the message body')
        body = user_commands()
        result = send_email(sender_email, sender_password, receiver_email, subject, body)
        engine_talk(result)

    elif 'ask' in command or 'query' in command or 'question' in command:
        engine_talk("What would you like to ask?")
        query = user_commands()
        engine_talk("Thinking...")
        answer = ask_chatgpt(query)
        print("ChatGPT:", answer)
        engine_talk(answer)

    elif 'screenshot' in command:
        result = take_screenshot()
        print(result)
        engine_talk(result)

    elif 'note' in command or 'reminder' in command or 'sticky' in command:
        engine_talk("What should I remind you?")
        note = user_commands()
        if note:
            result = create_sticky_note(note)
            engine_talk(result)
        else:
            engine_talk("Sorry, I didn’t catch that. Try again.")

    elif 'whatsapp' in command and 'message' in command:
        engine_talk("Who do you want to message?")
        contact = user_commands()
        if contact:
            engine_talk(f"What message should I send to {contact}?")
            msg = user_commands()
            if msg:
                send_whatsapp_message(contact, msg)
            else:
                engine_talk("Sorry, I missed the message.")
        else:
            engine_talk("Sorry, I missed the contact name.")

    else:
        engine_talk("I could not hear you properly.")


In [29]:
# Continuously runs the assistant
while True:
   run_alexa()

Start Speaking!!
Vani: yes, how can I help you?
You said:  play a song
Vani: Playing   a song
Start Speaking!!
Vani: Sorry, I missed that. Could you please repeat?
Start Speaking!!
Vani: yes, how can I help you?
You said:  tell me the weather
Vani: Please tell the name of the city.
Start Speaking!!
You said: gwalior
Vani: The current temperature in gwalior is 29.88°C with overcast clouds.
Start Speaking!!
Vani: yes, how can I help you?
You said:  send a whatsapp
Vani: I could not hear you properly.
Start Speaking!!
You said: send a whatsapp message
Vani: Who do you want to message?
Start Speaking!!
You said: baba
Vani: What message should I send to baba?
Start Speaking!!
You said: hello
Vani: Opening WhatsApp Web to message baba
Vani: Bringing WhatsApp Web to front...
 Message sent to: baba
Vani: Message sent successfully.
Start Speaking!!
You said: stop
Vani: Goodbye!


SystemExit: 

C:\Users\Asus\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
